In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import config
config.experimental_run_functions_eagerly(True)

from tensorflow.keras.preprocessing.image import load_img , ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Conv2D , MaxPooling2D , BatchNormalization , Flatten , Dropout , InputLayer

import tensorflow as tf

In [20]:
img_dir = 'dataset/Brain Tumor Data Set/'

In [21]:
BATCH_SIZE = 64
IMAGE_SIZE = 150
input_shape = (150,150,1)

In [22]:
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [23]:
train_gen = data_gen.flow_from_directory(img_dir,
                                        target_size=(IMAGE_SIZE , IMAGE_SIZE),
                                        batch_size=BATCH_SIZE,
                                        color_mode='grayscale',
                                        shuffle=True,
                                        class_mode='binary',
                                        subset='training'
                                        )

Found 3681 images belonging to 2 classes.


In [24]:
val_gen = data_gen.flow_from_directory(img_dir,
                                        target_size=(IMAGE_SIZE , IMAGE_SIZE),
                                        batch_size=BATCH_SIZE,
                                        color_mode='grayscale',
                                        shuffle=True,
                                        class_mode='binary',
                                        subset='validation'
                                        )

Found 919 images belonging to 2 classes.


In [25]:
labels = train_gen.class_indices
classes = list(labels.keys())

print(classes)

['Brain Tumor', 'Healthy']


In [27]:
model = Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Conv2D(16,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(512 , activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10 , activation='relu'))
model.add(Dense(1 , activation='sigmoid'))

C:\Users\amin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [28]:
model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 16)        │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 41472)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      21,234,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,244,117 (81.04 MB)

 Trainable params: 21,244,117 (81.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
with tf.device('/gpu:0'):
    history = model.fit(train_gen,verbose=1,epochs=15,validation_data=val_gen,steps_per_epoch = 3681//64,validation_steps = 919//64)

Epoch 1/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 605ms/step - accuracy: 0.9737 - loss: 0.0836 - val_accuracy: 0.9665 - val_loss: 0.1011
Epoch 2/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9688 - loss: 0.0842 - val_accuracy: 1.0000 - val_loss: 0.0517
Epoch 3/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 34s 599ms/step - accuracy: 0.9894 - loss: 0.0469 - val_accuracy: 0.9900 - val_loss: 0.0375
Epoch 4/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 1.0000 - loss: 0.0288 - val_accuracy: 1.0000 - val_loss: 0.0153
Epoch 5/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 34s 600ms/step - accuracy: 0.9958 - loss: 0.0233 - val_accuracy: 0.9933 - val_loss: 0.0391
Epoch 6/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9688 - loss: 0.0894 - val_accuracy: 0.9565 - val_loss: 0.0522
Epoch 7/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 34s 601ms/step - accuracy: 0.9973 - loss: 0.0167 - val_accuracy: 0.9955 - val_loss: 0.0219
Epoch 8/15
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.0000 - loss: 0.0037 - val_accuracy: 1.0

In [ ]:
test_loss, test_acc   = model.evaluate(val_gen)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.save('model.h5')